# Data augmentation

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import torch
import torch.nn as nn
import multiprocessing
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from main import MNIST_dataset, MNIST_trainer

In [10]:
print("Torch version: ", torch.__version__)

####################################################################
# Set Device
####################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Torch version:  2.5.1+cu124
Device:  cuda


In [11]:
####################################################################
# DataLoader Class
####################################################################
da = transforms.Compose(
    [
        transforms.RandomRotation((10)),
        transforms.ElasticTransform(alpha=90.0),
        transforms.ToTensor(),
    ]
)
train_dataset = MNIST_dataset(partition="train", da_transform=da)
test_dataset = MNIST_dataset(partition="test")

batch_size = 100
num_workers = multiprocessing.cpu_count() - 1
print("Num workers", num_workers)

train_dataloader = DataLoader(
    train_dataset, batch_size, shuffle=True, num_workers=num_workers
)
test_dataloader = DataLoader(
    test_dataset, batch_size, shuffle=False, num_workers=num_workers
)


Loading MNIST  train  Dataset...
	Total Len.:  60000 
 --------------------------------------------------

Loading MNIST  test  Dataset...
	Total Len.:  10000 
 --------------------------------------------------
Num workers 11


In [12]:
####################################################################
# Neural Network Class
####################################################################


# Creating our Neural Network - Fully Connected
class Net(nn.Module):
    def __init__(
        self,
        sizes=[[784, 1024], [1024, 1024], [1024, 1024], [1024, 10]],
        criterion=None,
    ):
        super(Net, self).__init__()

        self.layers = nn.ModuleList()

        for i in range(len(sizes) - 1):
            dims = sizes[i]
            self.layers.append(nn.Linear(dims[0], dims[1]))
            self.layers.append(nn.BatchNorm1d(dims[1]))
            self.layers.append(nn.ReLU())

        dims = sizes[-1]
        self.classifier = nn.Linear(dims[0], dims[1])

        self.criterion = criterion

    def forward(self, x, y=None):
        for layer in self.layers:
            x = layer(x)
        x = self.classifier(x)

        if y != None:
            loss = self.criterion(x, y)
            return loss, x
        return x


####################################################################
# Training settings
####################################################################

# Training hyperparameters
criterion = nn.CrossEntropyLoss()
# Instantiating the network and printing its architecture
num_classes = 10
net = Net(
    sizes=[[784, 1024], [1024, 1024], [1024, 1024], [1024, num_classes]],
    criterion=criterion,
)
print(net)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print("Params: ", count_parameters(net))
optimizer = optim.SGD(net.parameters(), lr=0.1, weight_decay=1e-6, momentum=0.9)
epochs = 50

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, "min", patience=0, factor=0.9, threshold=0.001, cooldown=1, verbose=True
)

trainer = MNIST_trainer(
    net,
    train_dataloader,
    test_dataloader,
    optimizer,
    criterion,
    epochs,
    device,
    scheduler=scheduler,
    model_path="models/da5.pt",
)

Net(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=1024, bias=True)
    (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (classifier): Linear(in_features=1024, out_features=10, bias=True)
  (criterion): CrossEntropyLoss()
)
Params:  2919434


In [13]:
####################################################################
# Training
####################################################################

trainer.train()


---- Start Training ----


Test 0: 100%|██████████| 100/100 [00:00<00:00, 117.51batch/s]


	LR:  0.1
[Epoch 1] Train Loss: 0.008046 - Test Loss: 0.000915 - Train Accuracy: 77.67% - Test Accuracy: 97.01%


Test 1: 100%|██████████| 100/100 [00:00<00:00, 122.16batch/s]


	LR:  0.1
[Epoch 2] Train Loss: 0.003869 - Test Loss: 0.000657 - Train Accuracy: 87.69% - Test Accuracy: 97.70%


...

	LR:  0.01667718169966658
[Epoch 48] Train Loss: 0.000882 - Test Loss: 0.000184 - Train Accuracy: 97.23% - Test Accuracy: 99.35%



Test 48: 100%|██████████| 100/100 [00:00<00:00, 126.68batch/s]

	LR:  0.015009463529699923
[Epoch 49] Train Loss: 0.000856 - Test Loss: 0.000187 - Train Accuracy: 97.21% - Test Accuracy: 99.38%



Test 49: 100%|██████████| 100/100 [00:00<00:00, 123.78batch/s]

	LR:  0.015009463529699923
[Epoch 50] Train Loss: 0.000868 - Test Loss: 0.000198 - Train Accuracy: 97.16% - Test Accuracy: 99.33%

BEST TEST ACCURACY:  99.46  in epoch  46


In [14]:
####################################################################
# Load best weights
####################################################################

trainer.get_model()

Test 49: 100%|██████████| 100/100 [00:00<00:00, 123.71batch/s]

Final best acc:  99.46
